In [10]:
import pandas as pd
from models import base_res_net, small_res_net, efficient_net, pretrained_mobilenet, big_model
import tensorflow as tf
import tensorflow.compat.v1 as tfc
from sklearn.utils.class_weight import compute_class_weight

import random
from modeling import predict, predict_from_csv
from src import InputPipeline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Check GPU for tf

In [11]:
# Some GPU setup
# for documentation about using gpus refer to: https://www.tensorflow.org/install/pip#windows-wsl2

tf.keras.backend.clear_session()

device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
try:  # prevent a previous session from being alive
  sess.close() 
except:
  pass

tfc.enable_eager_execution()
gpu_options= tfc.GPUOptions(per_process_gpu_memory_fraction = 0.90)
sess = tfc.InteractiveSession(config=tfc.ConfigProto(gpu_options=gpu_options))

### Optional Stuff
- Here we compute classweights since the data is very imbalanced

In [3]:
train_df = pd.read_csv("../data/train_images_stratified.csv")

In [4]:
class_weights = compute_class_weight(class_weight = 'balanced', classes = train_df["label"].unique(), y=train_df["label"])
class_weights = dict(enumerate(class_weights))

## Configure Parameters

In [5]:
INPUT_SHAPE = (220,220,3)

CONF = {
  "learning_rate": 0.00005,
  "batch_size": 48,
  "epochs": 250,
  "loss_function": "sparse_categorical_crossentropy",
  "metric": "sparse_categorical_accuracy",
}

## Make Input Pipelines

In [6]:
# Input pipeline for subspecies
sub_species_input_pipeline = InputPipeline(splits=(0.85,0.0,0.15), channels=3, batch_size=CONF["batch_size"], size=INPUT_SHAPE[:2])
# This has a stratified split
sub_species_input_pipeline.make_stratified_train_dataset(
  train_ds_path= "../data/train_ds_images_stratified.csv",
  val_ds_path="../data/val_ds_images_stratified.csv",
)


# Input pipeline for species
species_input_pipeline = InputPipeline(splits=(0.85,0.0,0.15), channels=3, batch_size=CONF["batch_size"], size=INPUT_SHAPE[:2])
species_input_pipeline.make_train_datasets(directory="../data/train_images/species_classify")  # This doesnt have a stratified split


Datasets populated!
Found 15704 files belonging to 70 classes.
Using 13349 files for training.
Using 2355 files for validation.
Datasets populated!


## Training a Model

In [7]:
from modeling import train_classifier

In [8]:
# making the species classifier
train_classifier(
  model_name="../classifiers/subspecies_effnet_250_classifier",
  input_shape=INPUT_SHAPE,
  classes_to_classify=200,
  configuration=CONF,
  model=efficient_net, 
  train_dataset=sub_species_input_pipeline.train_dataset,
  validation_dataset=sub_species_input_pipeline.validation_dataset,
)

Epoch 1/250
213/213 [==============================] - 38s 118ms/step - loss: 13.0383 - sparse_categorical_accuracy: 0.0097 - val_loss: 12.0205 - val_sparse_categorical_accuracy: 0.0474
Epoch 2/250
213/213 [==============================] - 21s 98ms/step - loss: 11.7895 - sparse_categorical_accuracy: 0.0431 - val_loss: 10.9944 - val_sparse_categorical_accuracy: 0.1320
Epoch 3/250
213/213 [==============================] - 22s 105ms/step - loss: 10.8590 - sparse_categorical_accuracy: 0.0907 - val_loss: 10.1883 - val_sparse_categorical_accuracy: 0.2002
Epoch 4/250
213/213 [==============================] - 21s 98ms/step - loss: 10.1085 - sparse_categorical_accuracy: 0.1375 - val_loss: 9.4981 - val_sparse_categorical_accuracy: 0.2560
Epoch 5/250
213/213 [==============================] - 20s 95ms/step - loss: 9.4436 - sparse_categorical_accuracy: 0.1819 - val_loss: 8.8956 - val_sparse_categorical_accuracy: 0.3047
Epoch 6/250
213/213 [==============================] - 20s 95ms/step - loss:

INFO:tensorflow:Assets written to: ../classifiers/subspecies_effnet_250_classifier\assets


INFO:tensorflow:Assets written to: ../classifiers/subspecies_effnet_250_classifier\assets


Model saved successfully under: ../classifiers/subspecies_effnet_250_classifier


In [ ]:
# making the species classifier
train_classifier(
  model_name="../classifiers/species_efficient_net_classifier",
  input_shape=INPUT_SHAPE,
  classes_to_classify=70,
  configuration=CONF,
  model=efficient_net, 
  train_dataset=species_input_pipeline.train_dataset,
  validation_dataset=species_input_pipeline.validation_dataset,
)

## Predict Stuff

In [9]:
# No label adjustments needed afterwards!!!!!!
predict_from_csv(
  subspecies_classifier="../classifiers/subspecies_effnet_250_classifier",
  dataset="../data/test_images_path.csv",
  path="../data/test_images",
  size=INPUT_SHAPE,
)

predicting: 100%|██████████| 4000/4000 [07:32<00:00,  8.84it/s]

Saving to: ../data/test_images_sample_1701261285.091552.csv
